# 필수 라이브러리 불러오기

In [2]:
!pip install tqdm

In [3]:
# 기본 라이브러리
import os
import json
import glob
import shutil
from tqdm import tqdm
import random

# 데이터 분석 / 시각화
import numpy as np
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt

# YOLO
from ultralytics import YOLO

# PyTorch (디바이스 설정)
import torch

# device 설정

In [4]:
device = 0 if torch.cuda.is_available() else 'cpu'
print("사용 디바이스:", "GPU" if device==0 else "CPU (M1/M2)")

사용 디바이스: CPU (M1/M2)


# 랜덤 시드 설정

In [5]:
# 재현성 세팅
random.seed(42)
np.random.seed(42)
torch.manual_seed(42)

# 경로 설정

In [6]:
# 경로 설정
BASE_DIR = "/Users/apple/Downloads/codeit/project/ai06-level1-project"

TRAIN_IMG_DIR = f"{BASE_DIR}/train_images"
TRAIN_ANN_DIR = f"{BASE_DIR}/train_annotations"
YOLO_IMG_OUT = f"{BASE_DIR}/yolo_dataset/images/train"
YOLO_LABEL_OUT = f"{BASE_DIR}/yolo_dataset/labels/train"

os.makedirs(YOLO_IMG_OUT, exist_ok=True)
os.makedirs(YOLO_LABEL_OUT, exist_ok=True)

# JSON에서 category_id 전체 수집

In [7]:
json_paths = glob.glob(f"{TRAIN_ANN_DIR}/**/*.json", recursive=True)

category_ids = set()

for jp in tqdm(json_paths):
    with open(jp, "r") as f:
        data = json.load(f)

    if len(data["annotations"]) > 0:
        cid = data["annotations"][0]["category_id"]
        category_ids.add(cid)

category_ids = sorted(list(category_ids))
print("총 고유 category_id 개수:", len(category_ids))

100%|██████████| 1001/1001 [00:00<00:00, 6715.38it/s]

총 고유 category_id 개수: 56


# YOLO class index ↔ category_id 매핑 생성

In [8]:
cat2idx = {cid: idx for idx, cid in enumerate(category_ids)}
idx2cat = {idx: cid for idx, cid in enumerate(category_ids)}

with open("category_mapping.json", "w") as f:
    json.dump({"cat2idx": cat2idx, "idx2cat": idx2cat}, f, indent=2)

print("category_mapping.json 저장 완료!")

category_mapping.json 저장 완료!


# JSON → YOLO TXT 변환

In [9]:
for jp in tqdm(json_paths):
    with open(jp, "r") as f:
        data = json.load(f)

    img = data["images"][0]
    anns = data["annotations"]

    img_name = img["file_name"]
    img_path = f"{TRAIN_IMG_DIR}/{img_name}"

    # 이미지가 없으면 스킵
    if not os.path.exists(img_path):
        continue

    # bbox 없는 경우 스킵
    if len(anns) == 0:
        continue

    # 1개만 존재
    ann = anns[0]
    x, y, w, h = ann["bbox"]
    cid = ann["category_id"]
    cls = cat2idx[cid]  # YOLO class index

    iw, ih = img["width"], img["height"]

    # YOLO format
    xc = (x + w/2) / iw
    yc = (y + h/2) / ih
    ww = w / iw
    hh = h / ih

    # 저장
    label_path = f"{YOLO_LABEL_OUT}/{img_name.replace('.png', '.txt')}"
    with open(label_path, "w") as f:
        f.write(f"{cls} {xc} {yc} {ww} {hh}\n")

    shutil.copy(img_path, f"{YOLO_IMG_OUT}/{img_name}")

print("YOLO 변환 완료!")

100%|██████████| 1001/1001 [00:01<00:00, 709.77it/s]

YOLO 변환 완료!


# data.yaml 만들기

In [10]:
with open("category_mapping.json", "r") as f:
    mapping = json.load(f)

names = list(mapping["idx2cat"].values())

yaml_content = f"""
train: ./yolo_dataset/images/train
val: ./yolo_dataset/images/train
nc: {len(names)}
names: {names}
"""

with open("dataset.yaml", "w") as f:
    f.write(yaml_content)

print("dataset.yaml 생성 완료!")

dataset.yaml 생성 완료!


# YOLO 학습

In [ ]:
model = YOLO("yolov8n.pt")

model.train(
    data="dataset.yaml",
    epochs=20,
    imgsz=640,
    batch=8,
    device=device
)

Ultralytics 8.3.235 🚀 Python-3.10.19 torch-2.9.1 CPU (Apple M1)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=dataset.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=20, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train11, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, pose=12.0, pretrained=True, profile=False

# Experimentation — 검증 & 테스트 시각화
개별 이미지 시각화 코드

In [ ]:
def show_prediction(model, img_path):
    result = model.predict(img_path)[0]
    plotted = result.plot()
    plt.figure(figsize=(8, 8))
    plt.imshow(plotted)
    plt.axis("off")
    plt.show()

test_img = "train_images/임의파일.png"
show_prediction(model, test_img)

# 테스트 이미지 예측 후 CSV 생성

In [ ]:
model = YOLO("runs/detect/train/weights/best.pt")

with open("category_mapping.json") as f:
    mapping = json.load(f)

idx2cat = mapping["idx2cat"]

TEST_DIR = "./test_images"
test_imgs = sorted(os.listdir(TEST_DIR))

rows = []
ann_id = 1

for i, img_name in enumerate(test_imgs):
    img_path = f"{TEST_DIR}/{img_name}"
    result = model.predict(img_path, conf=0.001)[0]

    for box in result.boxes:
        cls = int(box.cls[0])
        score = float(box.conf[0])
        x1, y1, x2, y2 = box.xyxy[0].tolist()
        w, h = x2 - x1, y2 - y1

        rows.append([
            ann_id,
            i + 1,                # image_id
            idx2cat[str(cls)],    # 원본 category_id 사용
            x1, y1, w, h,
            score
        ])
        ann_id += 1

df = pd.DataFrame(rows, columns=[
    "annotation_id", "image_id", "category_id",
    "bbox_x", "bbox_y", "bbox_w", "bbox_h", "score"
])

df.to_csv("kaggle_submission.csv", index=False)
print("kaggle_submission.csv 생성 완료!")